In [1]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import math
import uuid

from datetime import datetime, timezone
from pathlib import Path

plt.style.use("ggplot")

#  Harmonize and Export River Water Chemistry Data

This notebook performs reconstruction, calculation, quality control, and export of river water chemistry data derived from NIVA’s database. It processes NetCDF datasets, computes derived parameters, filters outliers, attaches CF-compliant metadata, and exports the results as standardized NetCDF files.

### How to run the analysis?
1. Clone the AquaINFRA GitHub repo
2. Make sure the following dependencies are installed in the environment: xarray, pandas, numpy, matplotlib, pathlib
3. Define input files, metadata, and output files.
4. Run all cells sequentially.

### Which programming languages are used?
- Python

### How does the workflow look like?
1. Read raw NetCDF files
2. Convert to pandas DataFrames
3. Reconstruct missing values
4. Compute derived parameters
5. Visualize original vs derived variables
6. Apply outlier filtering using percentile thresholds
7. Convert to xarray.Dataset and assign time + station coordinates
8. Attach variable and global metadata
9. Export NetCDF files 

### Which functions are used and how are they connected?
- `plot_reconstruction()` – Plots original vs reconstructed variables
- `plot_scatter()` – Compares calculated vs observed values
- `plot_lines()` – Plots time series
- `detect_outliers_per_station()` – Identifies and replaces outliers using 0.05–99 percentile filtering
- `xr.Dataset.from_dataframe()` – Converts cleaned DataFrames to xarray.Dataset
- `to_netcdf()` – Saves final datasets with CF/ACDD metadata to NetCDF format


## 0. User inputs

In [2]:
# Define input files 
wc_raw_data_dir = Path("../../data/river/water_chemistry/raw")
wc_raw_data_file_paths = list(wc_raw_data_dir.glob("raw_riverchem_*.nc"))

# Define parameters to reconstruct
reconstruction_config = {
    "TOTN": {
        "fallback": "TOTN_EF_usikker",
        "units": "µg/l",
        "preserve_as": "TOTN_orig"
    },
    "SiO2": {
        "compute_from": "Si",
        "formula": "Si * 2.14",
        "units": "mg/l",
        "preserve_as": "SiO2_orig",
        "calc_temp_var": "SiO2_calc"
    }
}

# Define derived parameters
derivation_config = [
    {
        "operation": "scale",
        "target": "POC",
        "factor": 1 / 1000
    },
    {
        "operation": "sum",
        "target": "TOC_calc",
        "sources": ["POC", "DOC"]
    },
    {
        "operation": "fill_from_other",
        "target": "TDP",
        "expression": "TOTP - PP",
        "condition": "TDP.isna() & PP.notna()",
        "requires": ["TDP", "PP", "TOTP"]
    },
    {
        "operation": "fill_from_other",
        "target": "PP",
        "expression": "TOTP - TDP",
        "condition": "PP.isna() & TDP.notna()",
        "requires": ["TDP", "PP", "TOTP"]
    },
    {
        "operation": "rowwise_sum",
        "target": "DIN",
        "sources": ["NO3-N", "NH4-N"],
        "only_when_all_present": True
    },
    {
        "operation": "difference",
        "target": "DON",
        "expression": "TOTN - DIN",
        "requires": ["TOTN", "DIN"]
    },
    {
        "operation": "mask_date_before",
        "target": "TOC",
        "file_contains": "40355",
        "date_column": "sample_date",
        "before": "1998-01-01"
    }, 
    {
        "operation": "mask_date_before",
        "target": "DOC",
        "file_contains": "40355",
        "date_column": "sample_date",
        "before": "2016-01-01"
    },
    {
        "operation": "mask_date_before",
        "target": "DOC",
        "file_contains": "40352",
        "date_column": "sample_date",
        "before": "2016-01-01"
    }, 
    {
        "operation": "mask_date_before",
        "target": "DOC",
        "file_contains": "40356",
        "date_column": "sample_date",
        "before": "2016-01-01"
    }
]

# Define variables to plot 
plot_config = [
    {
        "type": "scatter",
        "x": "TOC",
        "y": "TOC_calc",
        "xlabel": "TOC",
        "ylabel": "TOC_calc",
        "title": "TOC vs TOC_calc",
        "unit": "mg/l",
        "required": ["TOC", "TOC_calc"],
        "subplot": [0, 0]
    },
    {
        "type": "line",
        "columns": ["TOC", "DOC", "POC", "TOC_calc"],
        "labels": ["TOC", "DOC", "POC", "TOC_calc"],
        "title": "TOC, DOC, POC, TOC_calc",
        "unit": "mg/l",
        "required": ["TOC", "DOC", "POC", "TOC_calc"],
        "subplot": [0, 1]
    },
    {
        "type": "line",
        "columns": ["TDP", "PP", "TOTP"],
        "labels": ["TDP", "PP", "TOTP"],
        "title": "TDP, PP, TOTP",
        "unit": "µg/l",
        "required": ["TDP", "PP", "TOTP"],
        "subplot": [1, 0]
    },
    {
        "type": "line",
        "columns": ["NO3-N", "NH4-N", "DIN"],
        "labels": ["NO3", "NH4-N", "DIN"],
        "title": "NO3, NH4-N, DIN",
        "unit": "µg/l",
        "required": ["NO3-N", "NH4-N", "DIN"],
        "subplot": [1, 1]
    },
    {
        "type": "line",
        "columns": ["TOTN", "DIN", "DON"],
        "labels": ["TOTN", "DIN", "DON"],
        "title": "TOTN, DIN, DON",
        "unit": "µg/l",
        "required": ["TOTN", "DIN", "DON"],
        "subplot": [2, 0]
    }
]


#
outlier_config = {
    "lower_quantile": 0.0005,
    "upper_quantile": 0.99
}

# Parameters metadata
pars_metadata_df = pd.DataFrame({
    "parameter_name": ["DOC", "Color", "NH4-N", "NO3-N", "TRP", "POC", "POC_calc", "TOC_calc", "SPM", "SiO2", "TOC", "TOTN", "TOTP", "TDP", "DIN", 
                       "DON", "PP", "TSM", "UV_Abs_254nm", "UV_Abs_410nm"],
    "unit":  ["mg/l C", "mg Pt/l", "µg/l", "µg/l", "µg/l", "mg/l", "mg/l", "mg/l", "mg/l", "mg/l", "mg/l", "µg/l", "µg/l", "µg/l P", "µg/l", 
              "µg/l", "µg/l P", "mg/l", "Abs/cm", "Abs/cm"]
})

# Define a mapping from parameter names to long names
standard_name_map = {
    "DOC": "Dissolved Organic Carbon",
    "Color": "Water Color",
    "NH4-N": "Ammonium-N",
    "NO3-N": "Nitrate-N",
    "TRP": "Total Reactive Phosphorus",
    "POC": "Particulate Organic Carbon",
    "TOC_calc": "Total Organic Carbon Calculated",
    "SPM": "Suspended Particulate Matter", 
    "SiO2": "Silicate", 
    "TOC": "Total Organic Carbon",
    "TOTN": "Total Nitrogen",
    "TOTP": "Total Phosphorus",
    "TDP": "Total Dissolved Phosphorus",
    "DIN": "Dissolved Inorganic Nitrogen",
    "DON": "Dissolved Organic Nitrogen",
    "PP": "Particulate Phosphorus",
    "TSM": "Total Suspended Matter",
    "UV_Abs_254nm": "Ultraviolet Absorbance 254nm",
    "UV_Abs_410nm": "Ultraviolet Absorbance 410nm",
}

# Comments for selected variables
var_comments = {
    "TOTN": "TOTN is reconstructed using Eurofins data (TOTN_EF_usikker), which are considered more uncertain than NIVA data. However, due to gaps in NIVA data from 2017–2021, Eurofins measurements were included to ensure completeness.",
    "SiO2": "SiO2 is reconstructed from Si using a conversion factor of 2.14 (based on molecular weights). Different methods were used (photometric vs ICP-MS), but results are expected to be comparable.",
    "TOC_calc": "TOC_calc is computed as POC + DOC, for comparison with lab-reported TOC. It is often more accurate due to rounding errors and turbidity effects in TOC analysis.",
    "DIN": "DIN is calculated as NO3-N + NH4-N, only when both values are available.",
    "DON": "DON is calculated as TOTN - DIN, only where both TOTN and DIN are available.",
    "PP": "PP is computed as TOTP - TDP for the period 2017–2020.",
    "TDP": "TDP is computed as TOTP - PP for the period 2021–2024.",
    "TRP": "TRP includes both Soluble Reactive Phosphorus (SRP) and Particulate Reactive Phosphorus (PRP) fractions."
}

global_metadata_config = {
    "naming_authority": "no.niva",
    "project": "AquaINFRA",
    "iso_topic_category": "inlandWaters",
    "featureType": "timeSeries",
    "spatial_representation": "point",
    "creator_type": "institution",
    "creator_institution": "Norwegian Institute for Water Research (NIVA)",
    "institution": "Norwegian Institute for Water Research (NIVA)",
    "institution_short_name": "NIVA",
    "creator_name": "Areti Balkoni",
    "creator_email": "areti.balkoni@niva.no",
    "creator_url": "https://www.niva.no/en/employees/areti-balkoni",
    "data_owner": "Norwegian Institute for Water Research",
    "processing_level": "Harmonized and cleaned",
    "Conventions": "CF-1.7, ACDD-1.3",
    "publisher_name": "Norwegian Institute for Water Research",
    "publisher_email": "miljoinformatikk@niva.no",
    "publisher_institution": "Norwegian Institute for Water Research",
    "publisher_url": "https://niva.no",
    "license": "http://spdx.org/licenses/CC-BY-4.0(CC-BY-4.0)",
    "keywords": "GCMDSK:EARTH SCIENCE > WATER QUALITY > CHEMISTRY, GCMDLOC:CONTINENT > EUROPE > NORWAY",
    "keywords_vocabulary": "GCMDSK:GCMD Science Keywords, GCMDLOC:GCMD Locations",
    "history": "Harmonized and cleaned using percentile-based filtering",
    "toc_calc_notes": (
        "Lab-reported TOC values may underestimate total organic carbon in turbid samples due to the lack of stirring in the autosampler, "
        "causing particles to settle. "
        "TOC and DOC are rounded to 0.1 mg/L, adding further uncertainty, while POC is reported to the nearest µg/L. "
        "Therefore, TOC_calc (POC + DOC) is often more reliable than lab-reported TOC, but all results should be interpreted considering analytical uncertainties."
    )
}

processed_namespace_uuid = uuid.UUID("a23b7946-1a42-4d4a-bb0d-cf5a6cfb5670") 

# Where to save 
fig_dir = Path("../../figures/quality_control")
fig_dir.mkdir(parents=True, exist_ok=True)

output_dir = Path("../../data/river/water_chemistry/cleaned")
output_dir.mkdir(parents=True, exist_ok=True)

## 1. Load data

In [3]:
initial_dataframes = []
for fp in wc_raw_data_file_paths:
    ds = xr.open_dataset(fp)
    df = ds.to_dataframe().reset_index() # Convert to dataframes for easier analysis
    
    # Make sure date is datetime
    if "sample_date" in df.columns:
        df["sample_date"] = pd.to_datetime(df["sample_date"], errors="coerce")
    elif "time" in df.columns:
        df["time"] = pd.to_datetime(df["time"], errors="coerce")

    initial_dataframes.append((fp.name, df))

## 2. Reconstruct TOTN and SiO2 time series 

In [4]:
def plot_reconstruction(df, date_col, var_orig, var_final,
                        label_base="", station_label="", units="", output_path=None, colors=None):
    colors = colors or {'original': '#1b9e77', 'final': '#d95f02'}

    if var_orig not in df.columns or var_final not in df.columns:
        return

    plt.figure(figsize=(10, 4))
    plt.plot(df[date_col], df[var_orig], label=f'{label_base} (original)',
             color=colors['original'], marker='o', linestyle='none')
    plt.plot(df[date_col], df[var_final], label=f'{label_base} (reconstructed)',
             color=colors['final'], marker='.', linestyle='-', alpha=0.7)
    plt.title(f"{station_label}")
    plt.ylabel(f"{label_base} ({units})" if units else label_base)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()

    if output_path:
        plt.savefig(output_path)
        plt.close()
    else:
        plt.show()

In [5]:
reconstructed_dataframes = []

for fname, df in initial_dataframes:
    df = df.copy()
    date_col = "sample_date" if "sample_date" in df.columns else None
    if not date_col:
        continue

    # Station info
    station_name = df['station_name'].dropna().unique()
    station_id = df['station_id'].dropna().unique()
    station_str = station_name[0] if len(station_name) > 0 else fname
    station_id_str = f" ({station_id[0]})" if len(station_id) > 0 else ""
    station_label = f"{station_str}{station_id_str}"

    # Track temp columns to drop later
    intermediate_cols = []

    for var, settings in reconstruction_config.items():
        fallback = settings.get("fallback")
        compute_from = settings.get("compute_from")
        formula = settings.get("formula")
        preserve_as = settings.get("preserve_as", f"{var}_orig")
        calc_temp_var = settings.get("calc_temp_var", f"{var}_calc")
        units = settings.get("units", "")

        if var in df.columns:
            df[preserve_as] = df[var]

        # Fill from fallback
        if var in df.columns and fallback and fallback in df.columns:
            df[var] = df[var].fillna(df[fallback])
            intermediate_cols.append(fallback)

        # Compute from another variable
        if compute_from and compute_from in df.columns:
            df[calc_temp_var] = eval(formula, {}, {compute_from: df[compute_from]})

            if var in df.columns:
                df[var] = df[var].fillna(df[calc_temp_var])
            else:
                df[var] = df[calc_temp_var]

            intermediate_cols.extend([compute_from, calc_temp_var])

        # Plot comparison
        if preserve_as in df.columns and var in df.columns:
            output_file = fig_dir / f"{fname.rstrip('.nc')}_{var}_reconstruction.png"
            plot_reconstruction(df, date_col, preserve_as, var,
                                label_base=var, station_label=station_label,
                                units=units, output_path=output_file)

        intermediate_cols.extend([preserve_as])

    # Clean up
    df = df.drop(columns=[col for col in intermediate_cols if col in df.columns])
    reconstructed_dataframes.append((fname, df))

## 3. Compute derived parameters

In [6]:
derived_dataframes = []

for fname, df in reconstructed_dataframes:
    df = df.copy()

    for rule in derivation_config:
        op = rule["operation"]
        target = rule.get("target")

        # Scaling
        if op == "scale" and target in df.columns:
            factor = rule["factor"]
            df[target] = df[target] * factor

        # Simple sum of columns
        elif op == "sum":
            sources = rule["sources"]
            if all(col in df.columns for col in sources):
                df[target] = df[sources[0]] + df[sources[1]]

        # Fill with relevant parameters  
        elif op == "fill_from_other":
            if all(col in df.columns for col in rule["requires"]):
                context = {col: df[col] for col in df.columns}
                cond = eval(rule["condition"], {}, context)
                df.loc[cond, target] = eval(rule["expression"], {}, context)

        # Sum if all values are present
        elif op == "rowwise_sum":
            sources = rule["sources"]
            if all(col in df.columns for col in sources):
                df[target] = df.apply(
                    lambda row: sum(row[col] for col in sources)
                    if all(pd.notna(row[col]) for col in sources)
                    else np.nan,
                    axis=1
                )

        # Difference of two columns
        elif op == "difference":
            if all(col in df.columns for col in rule["requires"]):
                df[target] = df[rule["requires"][0]] - df[rule["requires"][1]]

        # Apply mask before a specific date for specific files
        elif op == "mask_date_before":
            if rule["file_contains"] in fname and target in df.columns and rule["date_column"] in df.columns:
                mask = pd.to_datetime(df[rule["date_column"]]) < pd.Timestamp(rule["before"])
                df.loc[mask, target] = np.nan

    derived_dataframes.append((fname, df))

In [7]:
def plot_scatter(ax, x, y, xlabel, ylabel, title, unit=None):
    ax.scatter(x, y, alpha=0.5)
    if pd.notna(x).any() and pd.notna(y).any():
        lims = [min(x.min(), y.min()), max(x.max(), y.max())]
        ax.plot(lims, lims, 'r--')

    unit_str = f" ({unit})" if unit else ""
    ax.set_xlabel(f"{xlabel}{unit_str}")
    ax.set_ylabel(f"{ylabel}{unit_str}")
    ax.set_title(title)

def plot_lines(ax, df, columns, labels, title, unit=None):
    for col, label in zip(columns, labels):
        if col in df.columns:
            ax.plot(df['sample_date'], df[col], label=label)
    ax.legend()
    ax.set_title(title)
    if unit:
        ax.set_ylabel(f'{unit}')
    else:
        ax.set_ylabel('##')

In [8]:
for fname, df in derived_dataframes:
    station = df['station_name'].dropna().unique()
    title = station[0] if len(station) else fname

    fig, axs = plt.subplots(3, 2, figsize=(14, 16))
    fig.suptitle(f"Station: {title}", fontsize=16)

    plotted = False

    for plot in plot_config:
        if not all(col in df.columns for col in plot["required"]):
            continue
    
        row, col = plot["subplot"]
        plotted = True 
    
        if plot["type"] == "scatter":
            plot_scatter(
                axs[row, col],
                df[plot["x"]],
                df[plot["y"]],
                plot["xlabel"],
                plot["ylabel"],
                plot["title"],
                unit=plot.get("unit")
            )
    
        elif plot["type"] == "line":
            plot_lines(
                axs[row, col],
                df,
                plot["columns"],
                plot["labels"],
                plot["title"],
                unit=plot.get("unit")
            )

    axs[2, 1].axis('off')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    if plotted:
        output_file = fig_dir / f"{fname.rstrip('.nc')}_derived_pars.png"
        plt.tight_layout(rect=[0, 0.03, 1, 0.95])
        plt.savefig(output_file, dpi=300, bbox_inches='tight')
        plt.close()
    else:
        plt.close() 

## 4. Outlier removal

In [9]:
def detect_outliers_per_station(df, station_name, filename, variables=None, outlier_config=None):
    df = df.copy()

    if outlier_config is None:
        outlier_config = {"lower_quantile": 0.05, "upper_quantile": 0.95}

    q_low = outlier_config["lower_quantile"]
    q_high = outlier_config["upper_quantile"]

    if variables is None:
        variables = [
            col for col in df.columns
            if pd.api.types.is_numeric_dtype(df[col]) and col not in ['station_id', 'depth', 'longitude', 'latitude']
        ]

    variables = [var for var in variables if var in df.columns and not df[var].dropna().empty]
    n_vars = len(variables)
    if n_vars == 0:
        return df

    ncols = 2
    nrows = math.ceil(n_vars / ncols)
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(14, 4 * nrows), squeeze=False)
    fig.suptitle(f"{station_name} – Outlier Detection (Q{q_low*100:.2f}–Q{q_high*100:.2f})", fontsize=16)

    for idx, var in enumerate(variables):
        row, col = divmod(idx, ncols)
        ax = axs[row][col]

        series = df[var].dropna()
        p5 = series.quantile(q_low)
        p95 = series.quantile(q_high)
        outliers = (df[var] < p5) | (df[var] > p95)

        ax.scatter(df['sample_date'], df[var], label=var, color='gray', alpha=0.7)
        ax.scatter(df.loc[outliers, 'sample_date'], df.loc[outliers, var], color='red', label='Outliers')
        ax.set_title(var)
        ax.set_xlabel("Date")
        ax.set_ylabel(var)
        ax.legend()
        ax.grid(True)

        df.loc[outliers, var] = np.nan

    for i in range(n_vars, nrows * ncols):
        row, col = divmod(i, ncols)
        axs[row][col].axis('off')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    base_name = Path(filename).stem
    output_path = fig_dir / f"{base_name}_detected_outliers.png"
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

    return df


cleaned_dataframes = []
for fname, df in derived_dataframes: 
    station_name = df['station_name'].dropna().unique()
    station_str = station_name[0] if len(station_name) else fname
    cleaned_df = detect_outliers_per_station(df, station_str, fname, outlier_config=outlier_config)
    cleaned_dataframes.append((fname, cleaned_df))

## 5. Create datasets and assign metadata and global attributes

In [10]:
datasets = []

for fname, df in cleaned_dataframes:
    df = df.copy()
    
    # Make sure date is datetime
    df["sample_date"] = pd.to_datetime(df["sample_date"])
    df = df.set_index("sample_date")

    # Extract metadata from first row
    lat = float(df["latitude"].iloc[0])
    lon = float(df["longitude"].iloc[0])
    station_id = int(df["station_id"].iloc[0])
    station_code = str(df["station_code"].iloc[0])
    station_name = str(df["station_name"].iloc[0])
    station_type = str(df["station_type"].iloc[0])

    # Remove constant fields from dataframe to avoid them being treated as varying
    df_clean = df.drop(columns=["latitude", "longitude", "station_id", "station_code", "station_name", "station_type"])

    # Convert to xarray dataset
    ds = xr.Dataset.from_dataframe(df_clean)

    # Add attributes to sample date (time coordinate)
    ds = ds.assign_coords(sample_date=("sample_date", df.index))
    ds["sample_date"].attrs.update({"standard_name": "time", "long_name": "Time of measurement", "axis": "T"})

    # Add scalar coordinates
    ds = ds.assign_coords(
        latitude=xr.DataArray(lat, dims=(), attrs={"standard_name": "latitude", "long_name": "latitude", "units": "degree_north"}),
        longitude=xr.DataArray(lon, dims=(), attrs={"standard_name": "longitude", "long_name": "longitude", "units": "degree_east"})
    )
    ds = ds.set_coords(["latitude", "longitude"])

    # Add station info 
    ds["station_id"] = xr.DataArray(station_id, dims=())
    ds["station_code"] = xr.DataArray(station_code, dims=())
    ds["station_name"] = xr.DataArray(station_name, dims=(), attrs={"cf_role": "timeseries_id"})
    ds["station_type"] = xr.DataArray(station_type, dims=())

    # Add metadata to data variables
    for var in ds.data_vars:
        if var in ["station_id", "station_code", "station_name", "station_type"]:
            continue  # skip scalar metadata fields
    
        match = pars_metadata_df[pars_metadata_df["parameter_name"] == var]
    
        if not match.empty:
            row = match.iloc[0]
            ds[var].attrs["units"] = row["unit"]
            ds[var].attrs["parameter_name"] = row["parameter_name"]
            ds[var].attrs["long_name"] = standard_name_map.get(var, row["parameter_name"])
    
            # Assign comments 
            if var in var_comments:
                ds[var].attrs["comment"] = var_comments[var]
                       
    datasets.append((fname, ds))

# Metadata example
datasets[0][1]["TOTN"].attrs

{'units': 'µg/l',
 'parameter_name': 'TOTN',
 'long_name': 'Total Nitrogen',
 'comment': 'TOTN is reconstructed using Eurofins data (TOTN_EF_usikker), which are considered more uncertain than NIVA data. However, due to gaps in NIVA data from 2017–2021, Eurofins measurements were included to ensure completeness.'}

In [11]:
int_encoding = {"dtype": "int32", "_FillValue": -9999}

for fname, ds in datasets:
    # Extract metadata
    station_name = str(ds["station_name"].values.item())
    station_code = str(ds["station_code"].values.item())
    station_id = int(ds["station_id"].values.item())
    lat = float(ds["latitude"].values.item())
    lon = float(ds["longitude"].values.item())

    unique_id = str(uuid.uuid5(processed_namespace_uuid, str(station_id)))

    # Prepare dataset-specific attributes
    dataset_metadata = global_metadata_config.copy()
    dataset_metadata.update({
        "id": unique_id,
        "title": f"Cleaned water chemistry measurements at station {station_name}",
        "title_no": f"Rensede kjemiske målinger ved stasjon {station_name}",
        "summary": f"Cleaned long-term water chemistry monitoring at station {station_name} (code: {station_code})",
        "summary_no": f"Rensede, langsiktige vannkjemiske målinger ved stasjon {station_name} (kode: {station_code})",
        "date_created": datetime.now(timezone.utc).strftime("%Y-%m-%dT%H:%M:%SZ"),
        "time_coverage_start": np.datetime_as_string(ds.sample_date.min().values, unit="s", timezone="UTC"),
        "time_coverage_end": np.datetime_as_string(ds.sample_date.max().values, unit="s", timezone="UTC"),
        "geospatial_lat_min": lat,
        "geospatial_lat_max": lat,
        "geospatial_lon_min": lon,
        "geospatial_lon_max": lon,
    })

    # Assign attributes
    ds.attrs = dataset_metadata

    # Define encoding
    encoding = {
        "sample_date": {
            "dtype": "int32",
            "_FillValue": None,
            "units": "seconds since 1970-01-01 00:00:00",
        },
        "longitude": {"_FillValue": None},
        "latitude": {"_FillValue": None},
    }
    if "station_id" in ds:
        encoding["station_id"] = int_encoding

    # Save file
    output_path = output_dir / f"cleaned_riverchem_{station_id}_{datetime.now().strftime('%d-%m-%Y')}.nc"
    ds.to_netcdf(output_path, encoding=encoding)
    print(f"Saved NetCDF: {output_path}")


Saved NetCDF: ..\..\data\river\water_chemistry\cleaned\cleaned_riverchem_40352_23-06-2025.nc
Saved NetCDF: ..\..\data\river\water_chemistry\cleaned\cleaned_riverchem_40355_23-06-2025.nc
Saved NetCDF: ..\..\data\river\water_chemistry\cleaned\cleaned_riverchem_40356_23-06-2025.nc
